In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl
import logging

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))
sys.path.append(os.path.expanduser('..'))

from spinorama.compute_cea2034 import sound_power
from spinorama.load import graph_melt
from spinorama.graph import graph_params_default, contour_params_default

import datas.metadata as metadata
df = fl.load('../cache.parse_all_speakers.h5')
# print(df)

In [ ]:
def normalize2(dfu):
    dfm = dfu.copy()
    for c in dfu.columns:
        if c != "Freq" and c != "On Axis":
            dfm[c] -= dfu["On Axis"]
        if c == "180°":
            dfm.insert(1, "-180°", dfu["180°"] - dfu["On Axis"])
    dfm["On Axis"] = 0
    return dfm


def compute_contour(dfu):
    # normalize dB values wrt on axis                                                                                                       
    dfm = normalize2(dfu)

    # get a list of columns                                                                                                                 
    vrange = []
    for c in dfm.columns:
        if c != "Freq" and c != "On Axis":
            angle = int(c[:-1])
            vrange.append(angle)
    if c == "On Axis":
            vrange.append(0)

    # y are inverted for display                                                                                                            
    vrange = list(reversed(vrange))

    # melt                                                                                                                                  
    dfm = graph_melt(dfm)
    # compute numbers of measurements                                                                                                       
    nm = dfm.Measurements.nunique()
    nf = int(len(dfm.index) / nm)
    logging.debug("unique={:d} nf={:d}".format(nm, nf))
    # index grid on a log scale log 2 ±= 0.3                                                                                                
    hrange = np.logspace(1.0 + math.log10(2), 4.0 + math.log10(2), nf)
    # 3d mesh                                                                                                                               
    af, am = np.meshgrid(hrange, vrange)
    # since it is melted generate slices                                                                                                    
    az = np.array([dfm.dB[nf * i : nf * (i + 1)] for i in range(0, nm)])
    if af.shape != am.shape or af.shape != az.shape:
        logging.error(
            "Shape mismatch af={0} am={1} az={2}".format(af.shape, az.shape, am.shape)
        )
    return (af, am, az)




In [ ]:
def reshape(x, y, z, nscale):
    nx, _ = x.shape
    # expand x-axis and y-axis                                                                                                              
    lxi = [
        np.linspace(x[0][i], x[0][i + 1], nscale, endpoint=False)
        for i in range(0, len(x[0]) - 1)
    ]
    lx = [i for j in lxi for i in j] + [x[0][-1] for i in range(0, nscale)]
    nly = (nx - 1) * nscale + 1
    # keep order                                                                                                                            
    ly = []
    if y[0][0] > 0:
        ly = np.linspace(np.max(y), np.min(y), nly)
    else:
        ly = np.linspace(np.min(y), np.max(y), nly)

    # on this axis, cheat by 1% to generate round values that are better in legend                                                          
    # round off values close to those in ykeep                                                                                              
    xkeep = [
        20,
        30,
        100,
        200,
        300,
        400,
        500,
        1000,
        2000,
        3000,
        4000,
        5000,
        10000,
        20000,
    ]

    def close(x1, x2, xkeep):
        for z in xkeep:
            if abs((x1 - z) / z) < 0.01 and z < x2:
                xkeep.remove(z)
                return z
        return x1

    lx2 = [close(lx[i], lx[i + 1], xkeep) for i in range(0, len(lx) - 1)]
    lx2 = np.append(lx2, lx[-1])
    # build the mesh                                                                                                                        
    rx, ry = np.meshgrid(lx2, ly)
    # copy paste the values of z into rz                                                                                                    
    rzi = np.repeat(z[:-1], nscale, axis=0)
    rzi_x, rzi_y = rzi.shape
    rzi2 = np.append(rzi, z[-1]).reshape(rzi_x + 1, rzi_y)
    rz = np.repeat(rzi2, nscale, axis=1)
    # print(rx.shape, ry.shape, rz.shape)                                                                                                   
    return (rx, ry, rz)


def compute_contour_smoothed(dfu, nscale=5):
    # compute contour                                                                                                                       
    x, y, z = compute_contour(dfu)
    if len(x) == 0 or len(y) == 0 or len(z) == 0:
        return (None, None, None)
    # std_dev = 1                                                                                                                           
    kernel = Gaussian2DKernel(1, 5)
    # kernel = RickerWavelet2DKernel(4)                                                                                                     
    # extend array by x5                                                                                                                    
    rx, ry, rz = reshape(x, y, z, nscale)
    # convolve with kernel                                                                                                                  
    rzs = ndimage.convolve(rz, kernel.array, mode="mirror")
    # return                                                                                                                                
    return (rx, ry, rzs)


In [ ]:
def graph_contour_common(af, am, az, graph_params):
    try:
        width = graph_params["width"]
        height = graph_params["height"]
        # more interesting to look at -3/0 range                                                                                            
        speaker_scale = None
        if "contour_scale" in graph_params.keys():
            speaker_scale = graph_params["contour_scale"]
        else:
            speaker_scale = contour_params_default["contour_scale"]
        #                                                                                                                                   
        colormap = "veridis"
        if "colormap" in graph_params:
            colormap = graph_params["colormap"]
        else:
            colormap = contour_params_default["colormap"]

        # flatten and build a Frame                                                                                                         
        freq = af.ravel()
        angle = am.ravel()
        db = az.ravel()
        if (freq.size != angle.size) or (freq.size != db.size):
            logging.debug(
                "Contour: Size freq={:d} angle={:d} db={:d}".format(
                    freq.size, angle.size, db.size
                )
            )
            return None

        source = pd.DataFrame({"Freq": freq, "Angle": angle, "dB": db})

        # tweak ratios and sizes                                                                                                            
        chart = alt.Chart(source)
        # classical case is 200 points for freq and 36 or 72 measurements on angles                                                         
        # check that width is a multiple of len(freq)                                                                                       
        # same for angles                                                                                                                   

        # build and return graph                                                                                                            
        logging.debug("w={0} h={1}".format(width, height))
        if width / source.shape[0] < 2 and height / source.shape[1] < 2:
            chart = chart.mark_point()
        else:
            chart = chart.mark_rect()

        chart = chart.transform_filter("datum.Freq>400").encode(
            alt.X("Freq:O", axis=None),
            alt.Y("Angle:O", title="Angle (deg.)", axis=None, sort=None),
            alt.Color(
                "dB:Q",
                scale=alt.Scale(scheme=colormap, domain=speaker_scale, nice=True),
            ),
        )
        return (chart + graph_empty(400, 20000, -180, 180)).properties(
            width=width, height=height
        )
    except KeyError as ke:
        logging.warning("Failed with {0}".format(ke))
        return None


def graph_contour(df, graph_params):
    af, am, az = compute_contour(df)
    if af is None or am is None or az is None:
        logging.error("contour is None")
        return None
    return graph_contour_common(af, am, az, graph_params)

In [ ]:
params = contour_params_default
params['width'] = 400
params['height'] = 400
params['contour_scale'] = [-21, -18, -15, -12, -9, -6, -3, 0]
#params['contour_scale'] = [-12, -9, -6, -3, -2, -1, 0, 3]

def compare(df1, title1, df2, title2):
    V1 = resample(df1['SPL Vertical_unmelted'], 400)
    V2 = resample(df2['SPL Vertical_unmelted'], 400)
    GV1 = graph_contour(V1, params).properties(title='{0} Vertical'.format(title1))
    GV2 = graph_contour(V2, params).properties(title='{0} Vertical'.format(title2))
    H1 = resample(df1['SPL Horizontal_unmelted'], 400)
    H2 = resample(df2['SPL Horizontal_unmelted'], 400)
    GH1 = graph_contour(H1, params).properties(title='{0} Horizontal'.format(title1))
    GH2 = graph_contour(H2, params).properties(title='{0} Horizontal'.format(title2))
    return (GV1 | GV2) & (GH1 | GH2)

In [ ]:
from spinorama.compute_normalize import resample


df1 = df['KEF LS50']['ASR']['asr']
df2 = df['NHT SB2']['ASR']['asr']

compare(df1, 'LS50 ASR', df2, 'NHT SB2 ASR')


In [ ]:
df1 = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')
df2 = parse_graphs_speaker('Genelec', 'Genelec 8351A', 'princeton')

compare(df1, '8341A ASR', df2, '8351A 3D3A')

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

af, am, az = compute_contour(df1['SPL Vertical_unmelted'])

im = plt.imshow(az, interpolation='bilinear', cmap=cm.RdYlGn,
                origin='lower', extent=[-10, 10, -10, 10],
                vmax=abs(az).max(), vmin=-abs(az).max())

plt.show()


In [ ]:
from math import log10

df1 = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')
V1 = df1['SPL Vertical_unmelted']

def ERB(f):
    # return 21.4 + log10(1+0.00437*f)
    return 24.7*(1.+f*4.37/1000)

V1S = V1.copy()
V1S.Freq = V1.Freq.apply(ERB)

params = contour_params_default
params['width'] = 400
params['height'] = 400
params['contour_scale'] = [-21, -12, -9, -6, -3, -2, -1, 0, 1, 2, 3]
GV1  = graph_contour(V1, params)
GV1S = graph_contour(V1S, contour_params_default)
V1S
GV1|GV1S